<a href="https://colab.research.google.com/github/cherif1989/cbh-vp-gt/blob/main/Webscraping_societecom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import re
import time
from bs4 import BeautifulSoup as bs

In [ ]:

# authenticate
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())
from gspread_dataframe import set_with_dataframe
from googleapiclient import discovery

In [ ]:
## read data from google sheet
def read_gs(spreadsheet_id, range_):
  credentials = None
  service = discovery.build('sheets', 'v4', credentials=credentials)

  request = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_) #, valueRenderOption=value_render_option, dateTimeRenderOption=date_time_render_option)
  response = request.execute()
  data= response.get('values', [])
  values_df = pd.DataFrame(data)
  return values_df

  

In [ ]:
def clear_GS(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet."""

    # The ID and range of a sample spreadsheet.

    credentials = None
   

    service = discovery.build('sheets', 'v4', credentials=credentials)

    body = {'valueInputOption': "RAW"}

    request = service.spreadsheets().values().clear(
        spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME,
        body={}).execute()

In [ ]:
def write2GS(values, SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME):

    credentials = None

    service = discovery.build('sheets', 'v4', credentials=credentials)

    data = [
        {
            'range': SAMPLE_RANGE_NAME,
            'values': values
        },
    ]
    body = {'valueInputOption': "RAW", 'data': data}
    result = service.spreadsheets().values().batchUpdate(
        spreadsheetId=SAMPLE_SPREADSHEET_ID, body=body).execute()

In [ ]:
def get_societe_url_by_keyword(q):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4103.116 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,zh-TW;q=0.6',
        'Referer': 'https://www.societe.com/',
    }
    url = 'https://www.societe.com/cgi-bin/search?champs={}'.format(q)
    # etablissements

    if (len(str(url))<9):
      url = '0' + str(url)
    r = requests.get(url, headers=headers)
    time.sleep(10)

    return r

In [ ]:
def get_societe_url_by(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,zh-TW;q=0.6',
        'Referer': 'https://www.societe.com/',
    }
    # url = 'https://www.societe.com/cgi-bin/search?champs={}'.format(q)
    r = requests.get(url, headers=headers)
    time.sleep(8)
    return r

In [ ]:
def find_info(df):
  title = df.loc[:,0].tolist()
  info  = df.loc[:,1].tolist()
  N = len(title)
  adresse_company=None
  r_siret   = None
  tva_code=None
  for i in range(N):
    if 'Adresse postale' in title[i]:
      adresse_company = info[i]
    elif 'Numéro SIRET (siège)' in title[i]:
      r_siret   = ', '.join(re.findall('\d{14}',info[i]))
    elif 'Numéro TVA Intracommunautaire' in title[i]:
      tva_code=info[i].split(' ')[0]

  return [adresse_company , r_siret,tva_code]

In [ ]:
gs_id = '1BzymiidCIvsR1DXiDVEjcibId-Qsviz3y0OMxOKb5aE'
###read the sheet of inspection request to see the request send.
##try to retrieve the result.
df_societe= read_gs(gs_id, 'Siren_check_societe.com!A1:L')
new_header = df_societe.iloc[0] #grab the first row for the header
df_societe= df_societe[1:] #take the data less the header row
df_societe.columns = new_header #set the header row as the df header
df_societe=df_societe.reset_index(drop=True)
df_societe

,Merchant ID,Url Admin,Siren,Siret,Code TVA,Url Societe.com,Merchant name,First name,Merchant address,Merchant Activity,Code NAF,Status
0,6340404,https://admin.wkda.de/merchant/view/6340404,908647761,90864776100018,FR19908647761,https://www.societe.com/societe/mcp-auto-90864...,Kamel ZERROUB,Kamel,855 RTE DE CLAIRE MARE 27290 ECAQUELON,Entretien et réparation de véhicules automobil...,4520A,Active
1,6339957,https://admin.wkda.de/merchant/view/6339957,908868581,90886858100012,FR66908868581,https://www.societe.com/societe/saint-jean-aut...,Julie DELAUNAY,Julie,3 RUE NATIONALE SAINT-JEAN-SUR-COUESNO 35140 R...,En instance de chiffrement,0000Z,Active
2,6319559,https://admin.wkda.de/merchant/view/6319559,883328189,88332818900013,FR63883328189,https://www.societe.com/societe/touraine-locat...,M Aurelien GUILLARD,Aurelien,"ECO-LUXE AUTOMOBILES, 27 RUE DE L INGENIEUR MO...",Commerce de voitures et de véhicules automobil...,4511Z,Active
3,6341946,https://admin.wkda.de/merchant/view/6341946,,None,,None,None,None,None,None,None,None
4,6302310,https://admin.wkda.de/merchant/view/6302310,888232279,88823227900024,FR52888232279,https://www.societe.com/societe/easy-move-8882...,Sophian TRAIA,Sophian,CHE BARBE DE GAT 33130 BEGLES,Transports de voyageurs par taxis,4932Z,Active
...,...,...,...,...,...,...,...,...,...,...,...,...
94,6165705,https://admin.wkda.de/merchant/view/6165705,481402998,48140299800035,FR62481402998,https://www.societe.com/societe/monsieur-najim...,MONSIEUR NAJIM LOUALI,NAJIM,"LN TRANSPORT, 12 ALL ROLAND DE LASSUS 44300 NA...",Transports de voyageurs par taxis,4932Z,Active
95,6347214,https://admin.wkda.de/merchant/view/6347214,None,None,None,None,None,None,None,None,None,None
96,6347225,https://admin.wkda.de/merchant/view/6347225,None,None,None,None,None,None,None,None,None,None
97,6347228,https://admin.wkda.de/merchant/view/6347228,None,None,None,None,None,None,None,None,None,None


In [ ]:
df_societe['Url Societe.com']=df_societe['Url Societe.com'].astype(str)

In [ ]:
###webscrap the url of merchant page on societe.com
df_societe['Url Societe.com']=df_societe['Url Societe.com'].astype(str)
df_societe['Siren']=df_societe['Siren'].astype(str)
for i in range(len(df_societe)):
  if df_societe['Siren'][i]!='None' and df_societe['Siren'][i]!="":
    if df_societe['Url Societe.com'][i]=="" or df_societe['Url Societe.com'][i]=="None":
      siren = df_societe['Siren'][i]
      print(siren)
      r = get_societe_url_by_keyword(siren)
      print(r)
      url = r.url
      if url=='https://www.societe.com/cgi-bin/search?champs='+str(siren):
        df_societe['Url Societe.com'][i]='Not found'
      else:
        df_societe['Url Societe.com'][i]=url

809936370
<Response [200]>


In [ ]:
df_societe

,Merchant ID,Url Admin,Siren,Siret,Code TVA,Url Societe.com,Merchant name,First name,Merchant address,Merchant Activity,Code NAF,Status
0,6340404,https://admin.wkda.de/merchant/view/6340404,908647761,90864776100018,FR19908647761,https://www.societe.com/societe/mcp-auto-90864...,Kamel ZERROUB,Kamel,855 RTE DE CLAIRE MARE 27290 ECAQUELON,Entretien et réparation de véhicules automobil...,4520A,Active
1,6339957,https://admin.wkda.de/merchant/view/6339957,908868581,90886858100012,FR66908868581,https://www.societe.com/societe/saint-jean-aut...,Julie DELAUNAY,Julie,3 RUE NATIONALE SAINT-JEAN-SUR-COUESNO 35140 R...,En instance de chiffrement,0000Z,Active
2,6319559,https://admin.wkda.de/merchant/view/6319559,883328189,88332818900013,FR63883328189,https://www.societe.com/societe/touraine-locat...,M Aurelien GUILLARD,Aurelien,"ECO-LUXE AUTOMOBILES, 27 RUE DE L INGENIEUR MO...",Commerce de voitures et de véhicules automobil...,4511Z,Active
3,6341946,https://admin.wkda.de/merchant/view/6341946,,None,,None,None,None,None,None,None,None
4,6302310,https://admin.wkda.de/merchant/view/6302310,888232279,88823227900024,FR52888232279,https://www.societe.com/societe/easy-move-8882...,Sophian TRAIA,Sophian,CHE BARBE DE GAT 33130 BEGLES,Transports de voyageurs par taxis,4932Z,Active
...,...,...,...,...,...,...,...,...,...,...,...,...
94,6165705,https://admin.wkda.de/merchant/view/6165705,481402998,48140299800035,FR62481402998,https://www.societe.com/societe/monsieur-najim...,MONSIEUR NAJIM LOUALI,NAJIM,"LN TRANSPORT, 12 ALL ROLAND DE LASSUS 44300 NA...",Transports de voyageurs par taxis,4932Z,Active
95,6347214,https://admin.wkda.de/merchant/view/6347214,None,None,None,None,None,None,None,None,None,None
96,6347225,https://admin.wkda.de/merchant/view/6347225,None,None,None,None,None,None,None,None,None,None
97,6347228,https://admin.wkda.de/merchant/view/6347228,None,None,None,None,None,None,None,None,None,None


In [ ]:
#### search the merchand info 
df_societe['Siret']=df_societe['Siret'].astype(str)
for i in range(len(df_societe)):
  if (df_societe['Siret'][i]=="None" or df_societe['Siret'][i]=="") and df_societe['Url Societe.com'][i]!="None" and df_societe['Url Societe.com'][i]!="" :
    try:
      url_merchant=df_societe['Url Societe.com'][i]
      r = get_societe_url_by(url_merchant)
      soup = bs(r.content, 'html.parser')
      ###put the webpage as an str and then search if the company is inscrite or not (active or not)
      page_str=str(soup)
      if 'INSCRITE' in page_str and "CESSATION"  not in page_str and 'Liquidation judiciaire' not in page_str:
        status = soup.find_all('span', 'sirenStatus active')[0].text
      else :
        status = 'Fermé'
      df_societe['Status'][i]=status
      tb = pd.read_html(r.content)[0]
      info = find_info(tb)
      df_societe['Siret'][i]=info[1]
      df_societe['Merchant address '][i]=info[0]
      df_societe['Code TVA'][i]=info[2]
      ###search the merchant info: merchant name
      merchant_infos=soup.find_all('a','ft-bold grey7')
      if len(merchant_infos)==1:
        name=str(merchant_infos[0])
        merchant_name=name.split('>')[1].replace('</a','')
      else:
        name=str(merchant_infos[1])
        merchant_name=name.split('>')[1].replace('</a','')
      df_societe['Merchant name'][i]=merchant_name
      merchant_names=merchant_name.split(" ")
      if  merchant_names[0]=="MONSIEUR" or  merchant_names[0]=="Monsieur" or  merchant_names[0]=="Madame" or  merchant_names[0]=="M" :
        first_name=merchant_names[1]
      else:
        first_name=merchant_names[0]
      df_societe['First name'][i]=first_name
      
      ###search the activity and the code naf
      activité_infos=soup.find_all('div','FicheDonnees__donneeGenerique')
      name=str(activité_infos[2])
      activité_info=name.split('>')[1].replace('\n','').replace('</div','')
      activité_info=activité_info.lstrip()
      activité=activité_info.split('(')[0]
      df_societe['Merchant Activity'][i]=activité
      code_NAF=activité_info.split('(')[1].replace(')','').replace('(','').replace(' ','')
      df_societe['Code NAF'][i]=code_NAF
    except:
      df_societe['Status'][i]="Fake"

In [ ]:
df_societe

,Merchant ID,Url Admin,Siren,Siret,Code TVA,Url Societe.com,Merchant name,First name,Merchant address,Merchant Activity,Code NAF,Status
0,6340404,https://admin.wkda.de/merchant/view/6340404,908647761,90864776100018,FR19908647761,https://www.societe.com/societe/mcp-auto-90864...,Kamel ZERROUB,Kamel,855 RTE DE CLAIRE MARE 27290 ECAQUELON,Entretien et réparation de véhicules automobil...,4520A,Active
1,6339957,https://admin.wkda.de/merchant/view/6339957,908868581,90886858100012,FR66908868581,https://www.societe.com/societe/saint-jean-aut...,Julie DELAUNAY,Julie,3 RUE NATIONALE SAINT-JEAN-SUR-COUESNO 35140 R...,En instance de chiffrement,0000Z,Active
2,6319559,https://admin.wkda.de/merchant/view/6319559,883328189,88332818900013,FR63883328189,https://www.societe.com/societe/touraine-locat...,M Aurelien GUILLARD,Aurelien,"ECO-LUXE AUTOMOBILES, 27 RUE DE L INGENIEUR MO...",Commerce de voitures et de véhicules automobil...,4511Z,Active
3,6341946,https://admin.wkda.de/merchant/view/6341946,,None,,None,None,None,None,None,None,None
4,6302310,https://admin.wkda.de/merchant/view/6302310,888232279,88823227900024,FR52888232279,https://www.societe.com/societe/easy-move-8882...,Sophian TRAIA,Sophian,CHE BARBE DE GAT 33130 BEGLES,Transports de voyageurs par taxis,4932Z,Active
...,...,...,...,...,...,...,...,...,...,...,...,...
94,6165705,https://admin.wkda.de/merchant/view/6165705,481402998,48140299800035,FR62481402998,https://www.societe.com/societe/monsieur-najim...,MONSIEUR NAJIM LOUALI,NAJIM,"LN TRANSPORT, 12 ALL ROLAND DE LASSUS 44300 NA...",Transports de voyageurs par taxis,4932Z,Active
95,6347214,https://admin.wkda.de/merchant/view/6347214,None,None,None,None,None,None,None,None,None,None
96,6347225,https://admin.wkda.de/merchant/view/6347225,None,None,None,None,None,None,None,None,None,None
97,6347228,https://admin.wkda.de/merchant/view/6347228,None,None,None,None,None,None,None,None,None,None


In [ ]:
df_out=df_societe.loc[:,['Siret','Code TVA','Url Societe.com','Merchant name','First name','Merchant address ','Merchant Activity','Code NAF','Status']]
df_out

,Siret,Code TVA,Url Societe.com,Merchant name,First name,Merchant address,Merchant Activity,Code NAF,Status
0,90864776100018,FR19908647761,https://www.societe.com/societe/mcp-auto-90864...,Kamel ZERROUB,Kamel,855 RTE DE CLAIRE MARE 27290 ECAQUELON,Entretien et réparation de véhicules automobil...,4520A,Active
1,90886858100012,FR66908868581,https://www.societe.com/societe/saint-jean-aut...,Julie DELAUNAY,Julie,3 RUE NATIONALE SAINT-JEAN-SUR-COUESNO 35140 R...,En instance de chiffrement,0000Z,Active
2,88332818900013,FR63883328189,https://www.societe.com/societe/touraine-locat...,M Aurelien GUILLARD,Aurelien,"ECO-LUXE AUTOMOBILES, 27 RUE DE L INGENIEUR MO...",Commerce de voitures et de véhicules automobil...,4511Z,Active
3,None,,None,None,None,None,None,None,None
4,88823227900024,FR52888232279,https://www.societe.com/societe/easy-move-8882...,Sophian TRAIA,Sophian,CHE BARBE DE GAT 33130 BEGLES,Transports de voyageurs par taxis,4932Z,Active
...,...,...,...,...,...,...,...,...,...
94,48140299800035,FR62481402998,https://www.societe.com/societe/monsieur-najim...,MONSIEUR NAJIM LOUALI,NAJIM,"LN TRANSPORT, 12 ALL ROLAND DE LASSUS 44300 NA...",Transports de voyageurs par taxis,4932Z,Active
95,None,None,None,None,None,None,None,None,None
96,None,None,None,None,None,None,None,None,None
97,None,None,None,None,None,None,None,None,None


In [ ]:
write2GS(df_out.where((pd.notnull(df_out)), None).values.tolist(),  gs_id, 'Siren_check_societe.com!D2')